In [1]:
import os
import argparse
import json
import numpy as np

import torch
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader

from datetime import datetime
from tqdm import tqdm

from logger import Logger
from model import WaveGrad
from data import AudioDataset, MelSpectrogramFixed
from benchmark import compute_rtf

from utils import ConfigWrapper, show_message, str2bool

import numpy as np

import glob
import random

import time

In [2]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
def run_training(rank, config):
    show_message('Initializing logger...', verbose=config.verbose, rank=rank)
    logger = Logger(config, rank=rank)
    
    mel_fn = MelSpectrogramFixed(
        sample_rate=config.data_config.sample_rate,
        n_fft=config.data_config.n_fft,
        win_length=config.data_config.win_length,
        hop_length=config.data_config.hop_length,
        f_min=config.data_config.f_min,
        f_max=config.data_config.f_max,
        n_mels=config.data_config.n_mels,
        window_fn=torch.hann_window
    ).cuda()
    
    show_message('Initializing model...', verbose=config.verbose, rank=rank)
    model = WaveGrad(config).cuda()
    show_message(f'Number of WaveGrad parameters: {model.nparams}', verbose=config.verbose, rank=rank)

    show_message('Initializing optimizer, scheduler and losses...', verbose=config.verbose, rank=rank)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=config.training_config.lr)
    scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer,
        step_size=config.training_config.scheduler_step_size,
        gamma=config.training_config.scheduler_gamma
    )
    if config.training_config.use_fp16:
        scaler = torch.cuda.amp.GradScaler()

    show_message('Initializing data loaders...', verbose=config.verbose, rank=rank)
    train_dataset = AudioDataset(config, training=True)
    train_sampler = None
    train_dataloader = DataLoader(
        train_dataset, batch_size=config.training_config.batch_size,
        sampler=train_sampler, drop_last=True
    )

    if rank == 0:
        test_dataset = AudioDataset(config, training=False)
        test_dataloader = DataLoader(test_dataset, batch_size=1)
        test_batch = test_dataset.sample_test_batch(
            config.training_config.n_samples_to_test
        )

    if config.training_config.continue_training:
        show_message('Loading latest checkpoint to continue training...', verbose=config.verbose, rank=rank)
        model, optimizer, iteration = logger.load_latest_checkpoint(model, optimizer)
        epoch_size = len(train_dataset) // config.training_config.batch_size
        epoch_start = iteration // epoch_size
    else:
        iteration = 0
        epoch_start = 0

    # Log ground truth test batch
    if rank == 0:
        audios = {
            f'audio_{index}/gt': audio
            for index, audio in enumerate(test_batch)
        }
        logger.log_audios(0, audios)
        specs = {
            f'mel_{index}/gt': mel_fn(audio.cuda()).cpu().squeeze()
            for index, audio in enumerate(test_batch)
        }
        #logger.log_specs(0, specs)

    show_message('Start training...', verbose=config.verbose, rank=rank)
    try:
        for epoch in range(epoch_start, config.training_config.n_epoch):
            # Training step
            model.train()
            model.set_new_noise_schedule(
                init=torch.linspace,
                init_kwargs={
                    'steps': config.training_config.training_noise_schedule.n_iter,
                    'start': config.training_config.training_noise_schedule.betas_range[0],
                    'end': config.training_config.training_noise_schedule.betas_range[1]
                }
            )
            for batch in (
                tqdm(train_dataloader, leave=False) \
                if config.verbose and rank == 0 else train_dataloader
            ):
                model.zero_grad()

                batch = batch.cuda()
                mels = mel_fn(batch)
                
                if config.training_config.use_fp16:
                    with torch.cuda.amp.autocast():
                        loss = model.compute_loss(mels, batch)
                    scaler.scale(loss).backward()
                    scaler.unscale_(optimizer)
                else:
                    loss = model.compute_loss(mels, batch)
                    loss.backward()
                
                grad_norm = torch.nn.utils.clip_grad_norm_(
                    parameters=model.parameters(),
                    max_norm=config.training_config.grad_clip_threshold
                )

                if config.training_config.use_fp16:
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    optimizer.step()

                loss_stats = {
                    'total_loss': loss.item(),
                    'grad_norm': grad_norm.item()
                }
                logger.log_training(iteration, loss_stats, verbose=False)
                
                iteration += 1

            # Test step after epoch on rank==0 GPU
            if epoch % config.training_config.test_interval == 0 and rank == 0:
                model.eval()
                model.set_new_noise_schedule(
                    init=torch.linspace,
                    init_kwargs={
                        'steps': config.training_config.test_noise_schedule.n_iter,
                        'start': config.training_config.test_noise_schedule.betas_range[0],
                        'end': config.training_config.test_noise_schedule.betas_range[1]
                    }
                )
                with torch.no_grad():
                    # Calculate test set loss
                    test_loss = 0
                    for i, batch in enumerate(
                        tqdm(test_dataloader) \
                        if config.verbose and rank == 0 else test_dataloader
                    ):
                        batch = batch.cuda()
                        mels = mel_fn(batch)
                        test_loss_ = model.compute_loss(mels, batch)
                        test_loss += test_loss_
                    test_loss /= (i + 1)
                    loss_stats = {'total_loss': test_loss.item()}

                    # Restore random batch from test dataset
                    audios = {}
                    specs = {}
                    test_l1_loss = 0
                    test_l1_spec_loss = 0
                    average_rtf = 0

                    for index, test_sample in enumerate(test_batch):
                        test_sample = test_sample[None].cuda()
                        test_mel = mel_fn(test_sample.cuda())

                        start = datetime.now()
                        y_0_hat = model.forward(
                            test_mel, store_intermediate_states=False
                        )
                        y_0_hat_mel = mel_fn(y_0_hat)
                        end = datetime.now()
                        generation_time = (end - start).total_seconds()
                        average_rtf += compute_rtf(
                            y_0_hat, generation_time, config.data_config.sample_rate
                        )
                        
                        test_l1_loss += torch.nn.L1Loss()(y_0_hat, test_sample).item()
                        test_l1_spec_loss += torch.nn.L1Loss()(y_0_hat_mel, test_mel).item()

                        audios[f'audio_{index}/predicted'] = y_0_hat.cpu().squeeze()
                        specs[f'mel_{index}/predicted'] = y_0_hat_mel.cpu().squeeze()

                    average_rtf /= len(test_batch)
                    show_message(f'Device: GPU. average_rtf={average_rtf}', verbose=config.verbose)

                    test_l1_loss /= len(test_batch)
                    loss_stats['l1_test_batch_loss'] = test_l1_loss
                    test_l1_spec_loss /= len(test_batch)
                    loss_stats['l1_spec_test_batch_loss'] = test_l1_spec_loss

                    logger.log_test(iteration, loss_stats, verbose=config.verbose)
                    logger.log_audios(iteration, audios)
                    #logger.log_specs(iteration, specs)

                logger.save_checkpoint(
                    iteration,
                    model,
                    optimizer
                )
            if epoch % (epoch//10 + 1) == 0:
                scheduler.step()
    except KeyboardInterrupt:
        print('KeyboardInterrupt: training has been stopped.')
        return


In [4]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

file_list = glob.glob("../meow/*.wav")
random.shuffle(file_list)


with open("train.txt", 'w') as f:
    f.write("\n".join(sorted(file_list)))
    
with open("test.txt", 'w') as f:
    f.write("\n".join(file_list[:10]))

config = {
    "model_config": {
        "factors": [5, 4, 3, 2, 2],
        "upsampling_preconv_out_channels": 384,
        "upsampling_out_channels": [128, 128, 64, 32, 32],
        "upsampling_dilations": [
            [1, 2, 4, 8],
            [1, 2, 4, 4],
            [1, 2, 4, 4],
            [1, 2, 1, 2],
            [1, 2, 1, 2],
        ],
        "downsampling_preconv_out_channels": 32,
        "downsampling_out_channels": [128, 128, 256, 512],
        "downsampling_dilations": [[1, 2, 4], [1, 2, 4], [1, 2, 4], [1, 2, 4]],
    },
    "data_config": {
        "sample_rate": 8000,
        "n_fft": 512,
        "win_length": 512,
        "hop_length": 240,
        "f_min": 80.0,
        "f_max": 8000,
        "n_mels": 1
    },
    "training_config": {
        "logdir": "logs/default/" + str(time.time()),
        "continue_training": False,
        "train_filelist_path": "train.txt",
        "test_filelist_path": "test.txt",
        "batch_size": 96,
        "segment_length": 2400,
        "lr": 0.001,
        "grad_clip_threshold": 1,
        "scheduler_step_size": 1,
        "scheduler_gamma": 0.9,
        "n_epoch": 100000000,
        "n_samples_to_test": 4,
        "test_interval": 500,
        "use_fp16": True,
        "training_noise_schedule": {"n_iter": 1000, "betas_range": [1e-06, 0.01]},
        "test_noise_schedule": {"n_iter": 50, "betas_range": [1e-06, 0.01]},
    },
    "verbose": True,
}
config = ConfigWrapper(**config)

In [ ]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

n_gpus = torch.cuda.device_count()

run_training(0, config)

Initializing logger...
Initializing model...
Number of WaveGrad parameters: 5321153
Initializing optimizer, scheduler and losses...
Initializing data loaders...


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Start training...


C:\Python\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:100: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.16it/s]


Device: GPU. average_rtf=0.3469790084565412
Iteration: 5 | Losses: [5.617231369018555, 0.9868875294923782, 8.078983187675476]


 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:03<00:02,  1.21s/it]